In [1]:
# Suppress warnings 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import numpy as np
import pandas as pd
np.set_printoptions(linewidth=200)
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="darkgrid")

In [4]:
%%time
root = '../data/hapt_data_set/RawData/'
acc = pd.read_table(root + 'acc_exp01_user01.txt', sep=' ', header=None, names=['x', 'y', 'z'])
gyro = pd.read_table(root + 'gyro_exp01_user01.txt', sep=' ', header=None, names=['x', 'y', 'z'])

CPU times: user 12 ms, sys: 12 ms, total: 24 ms
Wall time: 24.1 ms


In [42]:
from scipy import signal
from numpy.linalg import norm

In [ ]:
class CreateFeatures():
    fs = 50  # Sampling frequency of sensor data.
    
    
    
    def __init__(self, acc, gyro):
        """
        Args:
            acc (DataFrame): Accelerometer 3-axial raw signals
            gyro (DataFrame): Gyroscope 3-axial raw signals
        """
        self.acc = acc
        self.gyro = gyro
        self.acc_seg = None
        self.gyro_seg = None
        
        
    def apply_median_filter(self, sensor_signal):
        """
        A median filter with window length 5 is applied to remove noise in signals.
        Args:
            sensor_signal (DataFrame): Raw signal
        Returns:
            retval (DataFrame): Median filtered signal
        """
        return sensor_signal.rolling(window=5, center=True, min_periods=1).median()
        
        
    def apply_low_pass_butterworth_filter(self, sensor_signal):
        """
        A 3rd order low pass Butterworth filter with a corner frequency of 20 Hz is applied to remove noise in signals.
        Args:
            sensor_signal (DataFrame): Raw signal
        Returns:
            retval (DataFrame): Butterworth filtered signal
        """       
        fc = 20  # cutoff frequency
        w = fc / (self.__class__.fs / 2)  # Normalize the frequency
        b, a = signal.butter(3, w, 'low')  # 3rd order low pass Butterworth filter
        
        return pd.DataFrame(signal.filtfilt(b, a, sensor_signal, axis=0), columns=['x', 'y', 'z'])  # Apply Butterworth filter
       
    
    def segment_signal(self, acc, gyro):
        """
        Sample sensor signals in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window).
        Args:
            acc (DataFrame): Raw acceleration signal
            gyro (DataFrame): Raw gyro signal
        Returns:
            acc_seg (list): List of segmented acceleration sigmal. Element is a segmented DataFrame.
            gyro_seg (list): List of segmented gyro sigmal. Element is a segmented DataFrame.
        """
        assert len(acc) == len(gyro), f'Length of acceleration signal ({len(acc)}) does not match to length of gyro signal ({len(gyro)}).'
        
        win_size = 128
        overlap_rate = 0.5
        acc_seg, gyro_seg = [], []
        
        for start_idx in range(0, len(acc) - win_size, int(win_size * overlap_rate)):
            acc_seg.append(acc.iloc[start_idx:start_idx + win_size].reset_index(drop=True))
            gyro_seg.append(gyro.iloc[start_idx:start_idx + win_size].reset_index(drop=True))

        return acc_seg, gyro_seg
        
    
    def remove_gravity(self, acc):
        """
        Separate acceleration signal into body and gravity acceleration signal.
        Another low pass Butterworth filter with a corner frequency of 0.3 Hz is applied.
        Args:
            acc (DataFrame): Segmented acceleration signal
        Returns:
            acc_body (DataFrame): Body acceleration signal
            acc_grav (DataFrame): Gravity acceleration signal
        """
        fc = 0.3  # cutoff frequency
        w = fc / (self.__class__fs / 2)  # Normalize the frequency
        b, a = signal.butter(3, w, 'low')  # 3rd order low pass Butterworth filter
        
        acc_grav = pd.DataFrame(signal.filtfilt(b, a, acc, axis=0), columns=['x', 'y', 'z'])  # Apply Butterworth filter
        
        # Substract gravity acceleration from acceleration sigal.
        acc_body = acc - acc_grav
        return acc_body, acc_grav
    
    
    def obtain_jerk_signal(self, sensor_signal):
        """
        Derive signal to obtain Jerk signals
        Args:
            sensor_signal (DataFrame)
        Returns:
            jerk_signal (DataFrame):
        """
        jerk_signal = sensor_signal.diff(periods=1)  # Calculate difference 
        jerk_signal.iloc[0] = jerk_signal.iloc[1]  # Fillna
        jerk_signal = jerk_signal / (1 / self.__class__.fs)  # Derive in time (1 / sampling frequency)
        return jerk_signal
    
    
    def obtain_magnitude(self, sensor_signal):
        """
        Calculate the magnitude of these three-dimensional signals using the Euclidean norm
        Args:
            sensor_signal (DataFrame): Three-dimensional signals
        Returns:
            retval (array): Magnitude of three-dimensional signals
        """
        return  norm(sensor_signal, ord=2, axis=1)
    
    
    def obtain_amplitude_spectrum(self, sensor_signal):
        """
        Obtain amplitude spectrum using Fast Fourier Transform (FFT).
        Args:
            sensor_signal (DataFrame): Time domain signals
        Returns:
            amp (array): Amplitude spectrum
        """
        N = len(sensor_signal)
        dt = 1 / self.__class__.fs  # 1 / sampling frequency (50Hz)
        t = np.arange(0, N * dt, dt)  # time axis
        freq = np.linspace(0, 1.0 / dt, N)  # frequency axis
        win = np.hamming(N)  # hamming window
        
        df = pd.DataFrame(columns=['x', 'y', 'z'])
        for axis in ['x', 'y', 'z']:
            df[axis] = sensor_signal[axis] * win

        F = np.fft.fft(df, axis=0)  # Apply FFT
        F = pd.DataFrame(F, columns=['x', 'y', 'z'])  # Convert array to DataFrame
        F = F[:N//2+1]  # Remove the overlapping part

        amp = np.abs(F)  # Obtain the amplitude spectrum
        amp = pd.DataFrame(amp, columns=['x', 'y', 'z'])  # Convert array to DataFrame

        amp = amp / N * 2
        amp.iloc[0] = amp.iloc[0] / 2
        
        return amp
    
    
    def estimate_variables(self, sensor_signal, dim=1):
        """
        Calculate statistic variables from signal.
        Args:
            sensor_signal (DataFrame)
            dim (int): Dimensin of sensor signal.
        Returns:
            variables (array):
                mean(): Mean value
                std(): Standard deviation
                mad(): Median absolute deviation 
                max(): Largest value in array
                min(): Smallest value in array
                sma(): Signal magnitude area
                energy(): Energy measure. Sum of the squares divided by the number of values. 
                iqr(): Interquartile range 
                entropy(): Signal entropy
                arCoeff(): Autorregresion coefficients with Burg order equal to 4
                correlation(): correlation coefficient between two signals
                maxInds(): index of the frequency component with largest magnitude
                meanFreq(): Weighted average of the frequency components to obtain a mean frequency
                skewness(): skewness of the frequency domain signal 
                kurtosis(): kurtosis of the frequency domain signal 
                bandsEnergy(): Energy of a frequency interval within the 64 bins of the FFT of each window.
                angle(): Angle between to vectors.
        """
        mean_values = sensor_signal.mean().values
        std_values = sensor_signal.std().values
        mad_values = 

In [45]:
df = acc.iloc[0:10]

In [53]:
df.mad()

x    0.017389
y    0.006861
z    0.012528
dtype: float64